## Aula 3 - Processamento de Linguagem Natural - Spacy e Word2Vec

Na aula de hoje iremos explorar os seguintes tópicos:
- Spacy
- Word2Vec


In [ ]:
!pip install spacy -U
!python -m spacy download pt_core_news_lg
!python -m spacy download pt_core_news_md
!python -m spacy download pt_core_news_sm
!python -m spacy download en_core_web_sm
# Instalando a biblioteca gensim para trabalhar com word2vec e doc2vec
!pip install -U gensim
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-08-30 00:10:36.083632: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 568.2 MB 18 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
2022-08-30 00:11:25.879872: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')
2022-08-30 00:11:36.846400: E tensorflow/stream_executor/cu

In [ ]:
# Importando o numpy para manipulação de vetores
import numpy as np
# Manipulação de tabelas
import pandas as pd
# Expressões regulares
import re
# Importando o unidecode
from unidecode import unidecode
# Biblioteca de processamento de linguagem natural
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
# Importanto o extrator de features de texto CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Para visualizar or dados
import matplotlib.pyplot as plt
import seaborn as sns
# Tipagem
from typing import List
# Importanto o gensim.downloader para baixar os dados da wiki
import gensim.downloader as api
# Baixando as stopwords
nltk.download('stopwords')
nltk.download('punkt')  # https://www.nltk.org/_modules/nltk/tokenize/punkt.html
nltk.download('rslp')  # Stemmer em português

# https://www.nltk.org/howto/wordnet.html
nltk.download('wordnet')

# NLTK 3.6.6 release: December 2021:
# support OMW 1.4, use Multilingual Wordnet Data from OMW with newer Wordnet versions
nltk.download('omw-1.4')

wv_wiki = api.load('glove-wiki-gigaword-300')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
# Carregando os dados de tweets sobre desastres naturais
tweets = pd.read_csv('./9.3.tweets.csv', index_col=0)

In [ ]:
# Explorando as colunas e dados de tweets
tweets.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Verficando o tipo de cada coluna
# e a presença de elementos nulos no conjunto de dados
tweets.info()
# A coluna text refere-se ao texto do tweet
# A coluna target se o tweet é referente a um desastre (1) ou não (0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 1 to 10873
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [ ]:
# Temos mais dados sobre não desastres do que de desastres
tweets['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# Classe para o pre-processamento de dados
class PreProcessPhrase:

  def remove_acentuacao(self, phrase: str, debug: bool = False) -> str:
    # Utilizando a biblioteca unidecode para remover acentuação de texto
    phrase_fmt = unidecode(phrase)
    if debug:
      print('`remove_acentuacao`: Frase original', phrase)
      print('`remove_acentuacao`: Frase formatada', phrase_fmt)

    # Retornando a frase formatada
    return phrase_fmt


  def remove_digits(self, phrase: str, debug: bool = False) -> str:
    # utilizando expressões regulares para remoção de digitos
    phrase_no_digits = re.sub(r'\d', '', phrase)

    # Se quisermos ligar o debug, mostre a frase original e transformada
    if debug:
      print('`remove_digits`: Texto original:', phrase)
      print('`remove_digits`: Texto sem digitos:', phrase_no_digits)

    # Retornando a frase sem digitos
    return phrase_no_digits

  def remove_special_char(self, phrase: str, debug: bool = False) -> str:
    # utilizando expressões regulares para remoção de caracteres especiais
    phrase_no_special_char = re.sub(r'[^a-zA-Z0-9]+', ' ', phrase)

    # Se quisermos ligar o debug, mostre a frase original e transformada
    if debug:
      print('`remove_special_char`: Texto original:', phrase)
      print('`remove_special_char`: Texto sem caracteres especiais:', phrase_no_special_char)

    # Retornando a frase sem digitos
    return phrase_no_special_char

  def word_lower(self, word: str, debug: bool = False) -> str:
    try:
      # Formatando a palavra em caixa baixa
      word_fmt = word.lower()

      # Se o debug for True, iremos imprimir a palavra original e transformada
      if debug:
        print('`word_lower`: Palavra Original:', word)
        print('`word_lower`: Palavra transformada:', word_fmt)

    except:
      # Caso a palavra não seja uma string levante um erro (TypeError) informando qual o tipo da palavra passada
      raise TypeError(f'Esperava uma `word` no tipo str, foi passado uma {type(word)}')

    # Retornando a palavra formatada
    return word_fmt

  def remove_stopwords(self, words: List[str], debug=False) -> List[str]:
    # Carregando as stopwords (inglês)
    stopwords_en = stopwords.words('english')
    # Criando uma váriavel que irá armazenar elementos limpos, que não estejam dentro das stopwords
    clean_words = []

    # Percorrendo cada palavra da nossa lista de palavras
    for word in words:
      # Verificando se a palavra não está presente das stopwords
      if word not in stopwords_en:
        # Se a palavra não é uma stopword adicionamos elas a váriavel clean_words
        clean_words.append(word)
      else:
        # Caso a palavra seja uma stopword e estamos no modo de debug (debug=True)
        if debug:
          # Imprimimos qual a palavra da lista words é uma stopword
          print(f'`remove_stopwords`: A palavra {word} está presente nas stopwords')
    return clean_words

  def tokenizer(self, phrase: str, debug: bool) -> List[str]:
    words = word_tokenize(phrase)
    if debug:
        print('`tokenizer`: Frase original:', phrase)
        print('`tokenizer`: tokens:', words)
    return words

  def stemmer(self, words: List[str], debug: bool = False) -> List[str]:
    # Inicializando o Porter Stemmer (inglês)
    stemmer = PorterStemmer()
    # Criando uma lista vazia para armazenar as palavras stem
    stem_words = []
    for word in words:
      # Pegando o stem de cada palavra
      s_word = stemmer.stem(word)
      # Adicionando essa palavra modificada a lista stem_words
      stem_words.append(s_word)

    if debug:
        print('`stemmer`: Tokens originais:', words)
        print('`stemmer`: Tokens transformadods:', stem_words)
    return stem_words

  def lemmatizer(self, words: List[str], debug: bool = False) -> List[str]:
    # Inicializando o WordNetLemmatizer (inglês)
    lemmatizer = WordNetLemmatizer()
    # Criando uma lista vazia para armazenar as palavras lemma
    lemm_words = []
    for word in words:
      # Pegando o lemma de cada palavra
      l_words = lemmatizer.lemmatize(word, pos='v')
      # Adicionando essa palavra modificada (lemma) a lista lemm_words
      lemm_words.append(l_words)

    if debug:
        print('`lemmatizer`: Tokens originais:', words)
        print('`lemmatizer`: Tokens transformadods:', lemm_words)
    return lemm_words


  def pipeline(self, phrase: str, methods: List[str], debug: bool = False):
    switcher = {
        'remove_acentuacao': self.remove_acentuacao,
        'remove_digits': self.remove_digits,
        'remove_special_char': self.remove_special_char,
        'word_lower': self.word_lower,
        'remove_stopwords': self.remove_stopwords,
        'tokenizer': self.tokenizer,
        'stemmer': self.stemmer,
        'lemmatizer': self.lemmatizer
    }
    for method in methods:
      # remove_stopwords
      if method == 'remove_stopwords':
        phrase = switcher[method](phrase, debug=debug)
        
      else:
        phrase = switcher[method](phrase, debug=debug)

    return phrase



In [ ]:
# Aplicando o preprocessamento de dados
# Instânciando o preprocessador de frases
preprocess = PreProcessPhrase()
# Definindo os passos da nossa pipeline
pipeline = [
    'remove_digits',
    'remove_special_char',
    'word_lower',
    'tokenizer',
    'remove_stopwords',
    'stemmer'
]
# Aplicando a pipeline de preprocessamento para cada documento (linha)
tweets["filtered_words"] = tweets['text'].apply(preprocess.pipeline, methods=pipeline)

# Normalmente depois do processamento juntamos as palavras novamente em uma só string
tweets['join_words'] = tweets['filtered_words'].apply(lambda x: ' '.join(x))

In [ ]:
# Definindo as features e targets
X = tweets['join_words']
y = tweets['target']

In [ ]:
# Importando o train_test_split para separar os dados de treino e teste
from sklearn.model_selection import train_test_split

# Separando os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                     y, 
                                                     test_size = 0.3, 
                                                     random_state = 42)

### N-gramas

In [ ]:
frase = "13.000 pessoas receberam ordens de evacuação por #incêndios na California"

In [ ]:
# Aplicando o preprocessamento de dados
# Instânciando o preprocessador de frases
preprocess = PreProcessPhrase()
pipeline = [
    'word_lower',
    'tokenizer',
    'remove_stopwords',
    'stemmer'
]
# Aplicando o preprocessamento na frase `frase`
tokens = preprocess.pipeline(frase, methods=pipeline)


In [ ]:
# Imprimindo os unigramas
tokens

['13.000',
 'pessoa',
 'receberam',
 'orden',
 'de',
 'evacuação',
 'por',
 '#',
 'incêndio',
 'na',
 'california']

In [ ]:
# Imprimindo os bigramas
print(list(nltk.bigrams(tokens)))

[('13.000', 'pessoa'), ('pessoa', 'receberam'), ('receberam', 'orden'), ('orden', 'de'), ('de', 'evacuação'), ('evacuação', 'por'), ('por', '#'), ('#', 'incêndio'), ('incêndio', 'na'), ('na', 'california')]


In [ ]:
# Imprimindo os trigramas
print(list(nltk.trigrams(tokens)))

[('13.000', 'pessoa', 'receberam'), ('pessoa', 'receberam', 'orden'), ('receberam', 'orden', 'de'), ('orden', 'de', 'evacuação'), ('de', 'evacuação', 'por'), ('evacuação', 'por', '#'), ('por', '#', 'incêndio'), ('#', 'incêndio', 'na'), ('incêndio', 'na', 'california')]


**Drops**

Utilize o CountVectorizer e verifique como ficam as features quando utilizamos unigramas e bigramas no dataset de tweets

In [ ]:
# Importanto o extrator de features de texto CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Cria apenas unigramas
cv_unigrama = CountVectorizer(
    # N-gram range (min e max), indica se queremos contar n-gramas
    # (1,1) significa apenas unigramas
    # (1,2) siginifica unigramas e bigramas

    ngram_range=(1,1), # PREENCHA AQUI

    # Se esse parâmetro for True, o resultado é binário (0, 1)
    # Se falso o retorno é a contagem da palavra na frase, sua ocorrência por frase
    binary=False
)

In [ ]:
# Aplique o fit_transform com o cv_unigramas nos dados de treino
X_train_cv_unigrama = cv_unigrama.fit_transform(X_train).todense() # PREENCHA AQUI

In [ ]:
# Verifique o tamanho dos dados (X_train_cv_unigrama), tanto o número de linhas como de features
print(X_train_cv_unigrama.shape)

(5329, 13803)


In [ ]:
# Imprima as dez primeiras palavras do vocabulário
print(cv_unigrama.get_feature_names_out()[:10]) # PREENCHA AQUI
# Imprima o tamanho do vocabulário
print(cv_unigrama.get_feature_names_out().shape) # PREENCHA AQUI

['aa' 'aaaa' 'aaaaaaallll' 'aaarrrgghhh' 'aac' 'aadzvsr' 'aal' 'aamir'
 'aan' 'aannnnd']
(13803,)


In [ ]:
# Cria unigramas e bigramas
cv_bigrama = CountVectorizer(
    # N-gram range (min e max), indica se queremos contar n-gramas
    # (1,1) significa apenas unigramas
    # (1,2) siginifica unigramas e bigramas

    ngram_range=(1, 2), # PREENCHA AQUI

    # Se esse parâmetro for True, o resultado é binário (0, 1)
    # Se falso o retorno é a contagem da palavra na frase, sua ocorrência por frase
    binary=False
)

In [ ]:

# Aplique o fit_transform com o cv_bigrama nos dados de treino
X_train_cv_bigrama = cv_bigrama.fit_transform(X_train) # PREENCHA AQUI

In [ ]:
# Verifique o tamanho dos dados (X_train_cv_bigrama), tanto o número de linhas como de features
print(X_train_cv_bigrama.shape)

(5329, 52162)


In [ ]:
# Imprima as dez primeiras palavras do vocabulário
print(cv_bigrama.get_feature_names_out()[:10]) # PREENCHA AQUI
# Imprima o tamanho do vocabulário
print(cv_bigrama.get_feature_names_out().shape) # PREENCHA AQUI

['aa' 'aa ayyo' 'aa batteri' 'aa mgm' 'aa near' 'aaaa' 'aaaa ok'
 'aaaaaaallll' 'aaaaaaallll even' 'aaarrrgghhh']
(52162,)


**Houve um grande aumento da dimensionalidade utilizando uni e bigramas**

Portanto o modelo pode sofrer da maldição da dimensionalidade, para evitar isso, iremos utilizar o word2vec e doc2vec!

---
Sobre lematização na Língua Portuguesa:

Iremos utilizar a biblioteca Spacy

**Drops**  
Além da palavra em si, pelo Spacy utilizar POO, há diversos outros atributos interessantes.  
O primeiro é o POS, Part of speech, o segundo é o `dep_` que representa a dependência sintática.

Para saber mais acesse: https://spacy.io/usage/linguistic-features

Crie dois laços:

O primeiro que imprima a partir de cada `Token` a sua string (`text`) e a sua POS (`pos_`).

O seguindo laço, a partir de cada `Token` a sua string (`text`) e a sua POS (`pos_`), e a sua dep (`dep_`).


In [ ]:
# Imprima o texto e o POS (`pos_`)
for word in doc:
  print(word...., word....)  # PREENCHA AQUI

In [ ]:
# Imprima o texto, o POS (`pos_`), e a dependência sintática (`dep_`)
for word in doc:
  print(word...., word...._, word...)  # PREENCHA AQUI

# Word2vec

Da documentação do Word2Vec temos:

> In case you missed the buzz, Word2Vec is a widely used algorithm based on neural networks, commonly referred to as “deep learning” (though word2vec itself is rather shallow). Using large amounts of unannotated plain text, word2vec learns relationships between words automatically. The output are vectors, one vector per word.

> Word2Vec is a more recent model that embeds words in a lower-dimensional vector space using a shallow neural network. The result is a set of word-vectors where vectors close together in vector space have similar meanings based on context, and word-vectors distant to each other have differing meanings. For example, strong and powerful would be close together and strong and Paris would be relatively far.

Nesse contexto, podemos observar que o Word2Vec utiliza redes neurais rasas (shallow neural networks), sendo que no Word2Vec temos duas configurações:

- Skip-gram
- CBOW (Continuous Bag-of-Words)


<img src="https://leimao.github.io/images/article/2019-08-23-Word2Vec-Classic/word2vec.png">


No caso do CBOW, tentamos prever a palavra $w_{(t)}$ com base nas palavras da sua vizinhança (a palavra do meio).

No caso do skip-gram, a partir da palavra $w_{(t)}$ tentamos prever as palavras na sua vizinhança.

A camada do meio (conhecida como hidden-layer) é a camada que iremos utilizar, e ela forma o tamanho de vetor que desejamos ter (veremos melhor a seguir).




Mas o que significa isso? Conseguimos extrair informações importantes do contexto que a palavra se encontra e agrupar palavras de acordo com esse contexto!

No caso de skip-grams
<img src="https://i.stack.imgur.com/fKkRF.png">


E para o CBOW

<img src="https://kavita-ganesan.com/wp-content/uploads/skipgram-vs-cbow-continuous-bag-of-words-word2vec-word-representation-1024x538.png">

Para saber mais:

https://www.youtube.com/watch?v=wvsE8jm1GzE

**Usar word2vec 10k, perplexity 25, lr, 10, procurar Austin, e one, drinks**

http://projector.tensorflow.org/

https://jalammar.github.io/illustrated-word2vec/

In [ ]:
class PreProcessPhrase:

  def remove_acentuacao(self, phrase: str, debug: bool = False) -> str:
    # Utilizando a biblioteca unidecode para remover acentuação de texto
    phrase_fmt = unidecode(phrase)
    if debug:
      print('`remove_acentuacao`: Frase original', phrase)
      print('`remove_acentuacao`: Frase formatada', phrase_fmt)

    # Retornando a frase formatada
    return phrase_fmt


  def remove_digits(self, phrase: str, debug: bool = False) -> str:
    # utilizando expressões regulares para remoção de digitos
    phrase_no_digits = re.sub(r'\d', '', phrase)

    # Se quisermos ligar o debug, mostre a frase original e transformada
    if debug:
      print('`remove_digits`: Texto original:', phrase)
      print('`remove_digits`: Texto sem digitos:', phrase_no_digits)

    # Retornando a frase sem digitos
    return phrase_no_digits

  def remove_special_char(self, phrase: str, debug: bool = False) -> str:
    # utilizando expressões regulares para remoção de caracteres especiais
    phrase_no_special_char = re.sub(r'[^a-zA-Z0-9]+', ' ', phrase)

    # Se quisermos ligar o debug, mostre a frase original e transformada
    if debug:
      print('`remove_special_char`: Texto original:', phrase)
      print('`remove_special_char`: Texto sem caracteres especiais:', phrase_no_special_char)

    # Retornando a frase sem digitos
    return phrase_no_special_char

  def word_lower(self, word: str, debug: bool = False) -> str:
    try:
      # Formatando a palavra em caixa baixa
      word_fmt = word.lower()

      # Se o debug for True, iremos imprimir a palavra original e transformada
      if debug:
        print('`word_lower`: Palavra Original:', word)
        print('`word_lower`: Palavra transformada:', word_fmt)

    except:
      # Caso a palavra não seja uma string levante um erro (TypeError) informando qual o tipo da palavra passada
      raise TypeError(f'Esperava uma `word` no tipo str, foi passado uma {type(word)}')

    # Retornando a palavra formatada
    return word_fmt

  def remove_stopwords(self, words: List[str], debug=False) -> List[str]:
    # Carregando as stopwords (inglês)
    stopwords_en = stopwords.words('english')
    # Criando uma váriavel que irá armazenar elementos limpos, que não estejam dentro das stopwords
    clean_words = []

    # Percorrendo cada palavra da nossa lista de palavras
    for word in words:
      # Verificando se a palavra não está presente das stopwords
      if word not in stopwords_en:
        # Se a palavra não é uma stopword adicionamos elas a váriavel clean_words
        clean_words.append(word)
      else:
        # Caso a palavra seja uma stopword e estamos no modo de debug (debug=True)
        if debug:
          # Imprimimos qual a palavra da lista words é uma stopword
          print(f'`remove_stopwords`: A palavra {word} está presente nas stopwords')
    return clean_words

  def tokenizer(self, phrase: str, debug: bool) -> List[str]:
    words = word_tokenize(phrase)
    if debug:
        print('`tokenizer`: Frase original:', phrase)
        print('`tokenizer`: tokens:', words)
    return words

  def stemmer(self, words: List[str], debug: bool = False) -> List[str]:
    # Inicializando o Porter Stemmer (inglês)
    stemmer = PorterStemmer()
    # Criando uma lista vazia para armazenar as palavras stem
    stem_words = []
    for word in words:
      # Pegando o stem de cada palavra
      s_word = stemmer.stem(word)
      # Adicionando essa palavra modificada a lista stem_words
      stem_words.append(s_word)

    if debug:
        print('`stemmer`: Tokens originais:', words)
        print('`stemmer`: Tokens transformadods:', stem_words)
    return stem_words

  def lemmatizer(self, words: List[str], debug: bool = False) -> List[str]:
    # Inicializando o WordNetLemmatizer (inglês)
    lemmatizer = WordNetLemmatizer()
    # Criando uma lista vazia para armazenar as palavras lemma
    lemm_words = []
    for word in words:
      # Pegando o lemma de cada palavra
      l_words = lemmatizer.lemmatize(word, pos='v')
      # Adicionando essa palavra modificada (lemma) a lista lemm_words
      lemm_words.append(l_words)

    if debug:
        print('`stemmer`: Tokens originais:', words)
        print('`stemmer`: Tokens transformadods:', lemm_words)
    return lemm_words


  def pipeline(self, phrase: str, methods: List[str], debug: bool = False):
    switcher = {
        'remove_acentuacao': self.remove_acentuacao,
        'remove_digits': self.remove_digits,
        'remove_special_char': self.remove_special_char,
        'word_lower': self.word_lower,
        'remove_stopwords': self.remove_stopwords,
        'tokenizer': self.tokenizer,
        'stemmer': self.stemmer,
        'lemmatizer': self.lemmatizer
    }
    for method in methods:
      # remove_stopwords
      if method == 'remove_stopwords':
        phrase = switcher[method](phrase, debug=debug)
        
      else:
        phrase = switcher[method](phrase, debug=debug)

    return phrase



In [ ]:
textos = \
["Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all",
"Forest fire near La Ronge Sask. Canada",
"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
"13,000 people receive #wildfires evacuation orders in California",
"Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school"]

In [ ]:
# Aplicando o preprocessamento de dados
# Instânciando o preprocessador de frases
preprocess = PreProcessPhrase()
# Definindo os passos da nossa pipeline
pipeline = [
    'remove_digits',
    'remove_special_char',
    'word_lower',
    'tokenizer',
    'remove_stopwords',
    'lemmatizer'
]
# Aplicando a pipeline de preprocessamento para cada documento (linha)
textos_processados = [' '.join(preprocess.pipeline(frase,methods=pipeline)) for frase in textos]

In [ ]:
# Podemos aplicar o CountVectorizer em novas frases
frase = "alaska resident asked shelter place notified officer evacuation shelter place order expected"


In [ ]:
# Ele realiza a contagem de uma dada palavra na frase
# portanto se a frase tiver múltiplas ocorrências percebemos
# o aumento da contagem na coluna de feature
frase = "resident asked shelter shelter shelter shelter shelter place notified officer alaska evacuation shelter place order expected"


Como mencionado anteriormente os documentos são complexos, e portanto pode aumentar muito a quantidade de features, principalmente pelo contexto (n-gramas).

Para resolver esse problema, iremos utilizar o Word2Vec

No exemplo anterior o modelo Word2Vec foi treinado com poucos documentos.

O Word2Vec fica interessante quando temos uma grande quantidade de documentos.

Como a seguir:

In [ ]:
# Carregando
tweets = pd.read_csv('./9.3.tweets.csv', index_col=0)

In [ ]:
# Aplicando o preprocessamento de dados
# Instânciando o preprocessador de frases
preprocess = PreProcessPhrase()
# Definindo os passos da nossa pipeline
pipeline = [
    'remove_digits',
    'remove_special_char',
    'word_lower',
    'tokenizer',
    'remove_stopwords',
    'stemmer'
]
# Aplicando a pipeline de preprocessamento para cada documento (linha)
tweets["filtered_words"] = tweets['text'].apply(preprocess.pipeline, methods=pipeline)

# Normalmente depois do processamento juntamos as palavras novamente em uma só string
tweets['join_words'] = tweets['filtered_words'].apply(lambda x: ' '.join(x))

In [ ]:
# Definindo o target e as Features
# Para o Word2Vec utilizamos os tokens!
X = tweets['filtered_words']
y = tweets['target']

In [ ]:
# Importando o `train_test_split` para separar os dados de treino e teste
from sklearn.model_selection import train_test_split

# Separando o conjunto de dados em treino e teste (30% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                     y, 
                                                     test_size = 0.3, 
                                                     random_state = 42)

**Drops**

Vamos juntar os passos acima em uma única função!

A entrada é o documento (`doc`), o modelo ('Word2Vec`), método, e o debug.

Preenchar o código abaixo para viabilizar a transformação de uma lista de tokens (List[str]) para números (vetor).

O método será a média (como visto acima), ou a soma.

In [ ]:
def doc2vec(phrase: List[str], model: Word2Vec, method: str, debug: bool = False) -> np.ndarray:
  
  # Se o metodo passado for invalido gere um erro
  if method not in ['soma', 'media']:
      raise KeyError(f'Metodo não implementado {method}, escolha entre "media" e "soma"')

  # Inicializando uma lista vazia para armazenar o resultado
  result_vec = []
  for token in phrase:
    # Verifique se o token está presente no vocabulário do model

    if token in ...:  # PREENCHA AQUI
      # Se estiver presente, pegue o vetor correpondente da palavra

      token_array = model...  # PREENCHA AQUI

      result_vec.append(token_array)

  # Verificando se a lista não é vazia
  # Caso ela não seja, ou seja pelo menos uma palavra está presente no vocabulário
  if result_vec:
    result_vec = np.asarray(result_vec)
    if method == 'media':
      result_vec =  ... # PREENCHA AQUI
    elif method == 'soma':
      result_vec = ... # PREENCHA AQUI

  # Caso nenhuma palavra do doc esteja presente no vocabulário
  else:
    # Criando um vetor com zeros!
    result_vec = np.zeros(model.vector_size)

  if debug:
    print(f'Frase: {phrase}')
    print(f'Vetor: {result_vec}')
  return result_vec

In [ ]:
# Testando a media 
doc_vec1 = doc2vec(doc1, model, method='media', debug=True)

In [ ]:
# Testando a soma 
doc_vec2 = doc2vec(doc2, model, method='soma', debug=True)

In [ ]:
# Passando um documento com uma palavra não existente no vocabulário
# Retorna um array de zeros de mesmo tamanho (500)
invalido_vec = doc2vec(['cachorrinha'], model, method='soma', debug=True)

In [ ]:
# Passando um método incorreto gera um erro
invalido_vec = doc2vec(['cachorrinha'], model, method='a', debug=True)

In [ ]:
# Vamos transformar os dados de treino e teste de tokens para features númericas
X_train

In [ ]:
random_seed = 42
# Importando os modelos de ensemble (Boosting e bagging)
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# Importanto o modelo de regressão logística
from sklearn.linear_model import LogisticRegression

# Importandos as métricas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve


# Criando a nossa classe base (que contêm métodos que serão herdados pelas classes filhas)
class BaseModel:
  # Adicionando um nome de modelo (acessado por self.model_name)
  model_name = None
  # Adicionando um modelo base
  model = None

  # Definindo um padrão (não necessário mas interessante as vezes)
  def fit(self):
    pass
  # Definindo um padrão (não necessário mas interessante as vezes)
  def predict(self):
    pass
  # Definindo um padrão (não necessário mas interessante as vezes)
  def predict_proba(self):
    pass

  # Criando nossa função de avaliação do modelo
  # Recebe os argumentos
  # X: features
  # y_true: valores targets reais
  def evaluate(self, X, y_true):
    # Fazendo a predição utilizando as features de entrada
    y_predict = self.model.predict(X)
    # Calculando a curva ROC
    fpr, tpr, thresholds = roc_curve(y_true, self.model.predict_proba(X)[:,1])
    # Calculando o ROC-AUC
    auc = roc_auc_score(y_true, y_predict)
    # Calculando a acurácia
    accuracy = accuracy_score(y_true, y_predict)
    # Calculando a precisão
    precision = precision_score(y_true, y_predict, average='weighted')
    # Calculando a revocação
    recall = recall_score(y_true, y_predict, average='weighted')

    # Calculando o F1-score
    f1 = f1_score(y_true, y_predict, average='weighted')
    # Inicializando o plot do gráfico de ROC
    # Adicionando a legenda sendo o nome do modelo e a AUC
    plt.plot(fpr, tpr, label=f'{self.model_name} ROC (AUC = {auc:.2f})')
    
    # Imprimindo dados do modelo:
    # Nome do modelo
    print(f"Model      : {self.model_name}")
    # A sua acurácia
    print(f"Accuracy   : {accuracy:.4f}")
    # A sua precisão
    print(f"Precision  : {precision:.4f}")
    # A sua revocação
    print(f"Recall     : {recall:.4f}")
    # O seu F1-score
    print(f"F1 - Score : {f1:.4f}")
    # A sua ROC-AUC
    print(f"ROC - AUC  : {auc:.4f}")
    # Imprimindo um divisor (para facilitar a visualização)
    print("======================")

    # Salvando os dados do modelo em um dicionário
    results = {
        "model": self.model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "roc-auc": auc
    }
    # Retornando o dicionário contendo os seus dados
    return results

# Construindo uma classe de RandomForest que herde o modelo base
class MoviesRandomForest(BaseModel):
  # Função de inicialização
  def __init__(self, random_seed: int = 42, debug: bool = False):
    self.model_name = 'Random Forest'
    self.model = RandomForestClassifier(random_state = random_seed)
    self.debug = debug

  def fit(self, X, y):
    if self.debug:
      print(f'Realizando o fit do modelo {self.model_name}')
    self.model.fit(X, y)
  def predict(self, X):
    if self.debug:
      print(f'Realizando o predict do modelo {self.model_name}')
    return self.model.predict(X)
  def predict_proba(self, X):
    if self.debug:
      print(f'Realizando o predict_proba do modelo {self.model_name}')
    return self.model.predict_proba(X)


class MoviesLogisticRegression(BaseModel):
# Função de inicialização
  def __init__(self, random_seed: int = 42, debug: bool = False):
    self.model_name = 'Logistic Regression'
    self.model =  LogisticRegression(random_state = random_seed, 
                                  solver = 'lbfgs')
    self.debug = debug


  def fit(self, X, y):
    if self.debug:
      print(f'Realizando o fit do modelo {self.model_name}')
    self.model.fit(X, y)
  def predict(self, X):
    if self.debug:
      print(f'Realizando o predict do modelo {self.model_name}')
    return self.model.predict(X)
  def predict_proba(self, X):
    if self.debug:
      print(f'Realizando o predict_proba do modelo {self.model_name}')
    return self.model.predict_proba(X)

class MoviesAdaBoost(BaseModel):
# Função de inicialização
  def __init__(self, random_seed: int = 42, debug: bool = False):
    self.model_name = 'Ada Boost'
    self.model = AdaBoostClassifier(random_state = random_seed)
    self.debug = debug


  def fit(self, X, y):
    if self.debug:
      print(f'Realizando o fit do modelo {self.model_name}')
    self.model.fit(X, y)
  def predict(self, X):
    if self.debug:
      print(f'Realizando o predict do modelo {self.model_name}')
    return self.model.predict(X)
  def predict_proba(self, X):
    if self.debug:
      print(f'Realizando o predict_proba do modelo {self.model_name}')
    return self.model.predict_proba(X)

def model_test_pipeline(models: List[str], X_train, X_test, y_train, y_test, debug: bool = False):
  model_switcher = {
      "MoviesRandomForest": MoviesRandomForest,
      "MoviesLogisticRegression": MoviesLogisticRegression,
      "MoviesAdaBoost": MoviesAdaBoost
  }
  final_results = []
  for model in models:
    selected_model = model_switcher[model](debug=debug)
    selected_model.fit(X_train, y_train)
    model_results = selected_model.evaluate(X_test, y_test)
    final_results.append(model_results)
  # Adicionando estilo ao plot
  plt.plot([0, 1], [0, 1], 'r--')
  plt.xlim(0, 1)
  plt.ylim(0, 1.05)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC-AUC curve')
  plt.legend(loc='lower right')
  plt.show()

  # Transformando os resultados em uma tabela
  results_df = pd.DataFrame(final_results)
  return results_df
